In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
import math
import statistics
from scipy import stats

In [ ]:
#runs an offline simulation using:
    #a directedness model
    #some initial ground truth data to begin making predictions
    #a specification of a time-varying EF
#and returns the entire predicted time series
#this function accepts four arguments:
    #model is a keras model that accepts two features (ef and dir) with a lookback of 20 timesteps
    #init is the initial ground truth data in the form of a dataframe (only the first 20 rows will be used)
    #ef_series is a np array with shape (n,2) where the first col is efs occuring for timesteps after 
        #the ground truth 20 timesteps
    #cols is the columns to use as input
#and returns the ef_series array with the dir column filled in
def run_sim(model, init, ef_series, cols=['volt', 'cum_dir']):
    ground = init[cols].to_numpy()
    ground = ground[:20,:]
    
    ef_series = np.concatenate((ground,ef_series), axis=0)
    
    for i in range(20,len(ef_series)):
        ef_series[i,1] = model.predict(ef_series[i-20:i,:].reshape(1,20,2))
    
    return ef_series

In [ ]:
df = pd.read_csv('data/cncc.csv')
df['volt'] = df['ef']/1000

dfr = pd.read_csv('data/cncc_reverse.csv')

df.head()

In [ ]:
ef_series = []
for i in range(16):
    ef_series.append(0)
for i in range(36):
    ef_series.append(.025)
for i in range(36):
    ef_series.append(.050)
for i in range(36):
    ef_series.append(.075)
for i in range(36):
    ef_series.append(.100)
for i in range(36):
    ef_series.append(.125)
for i in range(36):
    ef_series.append(.150)
for i in range(36):
    ef_series.append(.175)
for i in range(36):
    ef_series.append(.200)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print(ef_series)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0

model = keras.models.load_model('models/cncc_lstm.h5')
print('Model loaded')

fig, axs = plt.subplots()
ax2 = axs.twinx()


results = []

for track in range(1,51):
    print('Track: {}'.format(track))
    
    init = df.loc[(df['track']==track)&(df['ef']==0)&(df['set']==1)]

    result = run_sim(model, init, ef_series)
    results.append(result)
    
    color = 'tab:blue'
    axs.set_ylabel('Directedness', color=color)
    axs.plot(result[:,1], color=color)
    axs.tick_params(axis='y', labelcolor=color)

color = 'tab:orange'
ax2.set_title('Offline Simulation Increasing')
ax2.set_xlabel('Timesteps (5min/step)')
ax2.set_ylabel('EF (mV/mm)', color=color)
ax2.plot(results[0][:,0]*1000, color=color)
ax2.tick_params(axis='y', labelcolor=color)
    
avg_dirs = []
for i in range(len(results[0][:,0])):
    avg = 0
    for j in range(len(results)):
        result = results[j]
        avg = avg + result[i,1]
    avg = avg/len(results)
    avg_dirs.append(avg)
    
color = 'tab:red'
axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_increasing.pdf')

In [ ]:
ef_series = []
for i in range(16):
    ef_series.append(.2)
for i in range(37):
    ef_series.append(.175)
for i in range(37):
    ef_series.append(.150)
for i in range(37):
    ef_series.append(.125)
for i in range(37):
    ef_series.append(.1)
for i in range(37):
    ef_series.append(.075)
for i in range(37):
    ef_series.append(.050)
for i in range(37):
    ef_series.append(.025)
for i in range(37):
    ef_series.append(0)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print(ef_series)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0

model = keras.models.load_model('models/cncc_lstm.h5')
print('Model loaded')

fig, axs = plt.subplots()
ax2 = axs.twinx()


results = []

for track in range(1,51):
    print('Track: {}'.format(track))
    
    init = df.loc[(df['track']==track)&(df['ef']==200)&(df['set']==1)]

    result = run_sim(model, init, ef_series)
    results.append(result)
    
    color = 'tab:blue'
    axs.set_ylabel('Directedness', color=color)
    axs.plot(result[:,1], color=color)
    axs.tick_params(axis='y', labelcolor=color)

color = 'tab:orange'
ax2.set_title('Offline Simulation Decreasing')
ax2.set_xlabel('Timesteps (5min/step)')
ax2.set_ylabel('EF (mV/mm)', color=color)
ax2.plot(results[0][:,0]*1000, color=color)
ax2.tick_params(axis='y', labelcolor=color)
    
avg_dirs = []
for i in range(len(results[0][:,0])):
    avg = 0
    for j in range(len(results)):
        result = results[j]
        avg = avg + result[i,1]
    avg = avg/len(results)
    avg_dirs.append(avg)
    
color = 'tab:red'
axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_decreasing.pdf')

In [ ]:
ef_series = []
for i in range(17):
    ef_series.append(0.1)
for i in range(36):
    ef_series.append(0.0)
for i in range(36):
    ef_series.append(0.1)
for i in range(36):
    ef_series.append(0.0)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print(ef_series)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0

model = keras.models.load_model('models/cncc_lstm/model9.h5')
print('Model loaded')

fig, axs = plt.subplots()
ax2 = axs.twinx()


results = []

for track in range(1,51):
    print('Track: {}'.format(track))
    
    init = df.loc[(df['track']==track)&(df['ef']==100)&(df['set']==2)]

    result = run_sim(model, init, ef_series)
    results.append(result)
    
    color = 'tab:blue'
    axs.set_ylabel('Directedness', color=color)
    axs.plot(result[:,1], color=color)
    axs.tick_params(axis='y', labelcolor=color)

color = 'tab:orange'
ax2.set_title('Offline Simulation Pulse')
ax2.set_ylabel('EF (mV/mm)', color=color)
ax2.plot(results[0][:,0]*1000, color=color)
ax2.tick_params(axis='y', labelcolor=color)

axs.set_xlabel('Timesteps (5min/step)')
    
avg_dirs = []
for i in range(len(results[0][:,0])):
    avg = 0
    for j in range(len(results)):
        result = results[j]
        avg = avg + result[i,1]
    avg = avg/len(results)
    avg_dirs.append(avg)
    
color = 'tab:red'
axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_pulse.pdf')

In [ ]:
ef_series = []
for i in range(5):
    ef_series.append(.200)
for i in range(25):
    ef_series.append(-.200)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print(ef_series)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0

model = keras.models.load_model('models/cncc_lstm.h5')
print('Model loaded')

fig, axs = plt.subplots()
ax2 = axs.twinx()


results = []

for track in range(1,51):
    print('Track: {}'.format(track))
    
    init = df.loc[(df['track']==track)&(df['ef']==200)&(df['set']==1)]

    result = run_sim(model, init, ef_series)
    results.append(result)
    
    color = 'tab:blue'
    axs.set_ylabel('Directedness', color=color)
    axs.plot(result[:,1], color=color)
    axs.tick_params(axis='y', labelcolor=color)

color = 'tab:orange'
ax2.set_title('Offline Simulation Polarity Switch')
ax2.set_xlabel('Timesteps (5min/step)')
ax2.set_ylabel('EF (mV/mm)', color=color)
ax2.plot(results[0][:,0]*1000, color=color)
ax2.tick_params(axis='y', labelcolor=color)
    
avg_dirs = []
for i in range(len(results[0][:,0])):
    avg = 0
    for j in range(len(results)):
        result = results[j]
        avg = avg + result[i,1]
    avg = avg/len(results)
    avg_dirs.append(avg)
    
color = 'tab:red'
axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_reverse.pdf')

# Constant EF Simulations

Do simulations for each EF strength, and get final directedness values

Compare those with actual final directedness

In [ ]:
final_dirs = {}
all_res = {}
avg_dirs = {}

In [ ]:
final_dirs[0] = []
ef_series = []
for i in range(17):
    ef_series.append(0)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0
    
fig, axs = plt.subplots()
ax2 = axs.twinx()

for i in range(50):
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
    print('Model {} loaded'.format(i))

    final_dirs[0].append([])

    results = []

    for track in range(1,51):
        init = df.loc[(df['track']==track)&(df['ef']==0)&(df['set']==2)]

        result = run_sim(model, init, ef_series)
        results.append(result)
        final_dirs[0][i].append(result[-1,1])

        color = 'tab:blue'
        axs.set_ylabel('Directedness', color=color)
        #axs.plot(result[:,1], color=color)
        axs.tick_params(axis='y', labelcolor=color)

    all_res[0] = results
    
    axs.set_xlabel('Timesteps (5min/step)')
    color = 'tab:orange'
    ax2.set_title('Offline Simulation 0mV/mm')
    ax2.set_ylabel('EF (mV/mm)', color=color)
    ax2.plot(all_res[0][0][:,0]*1000, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,200])

    avg_dirs[(0,i)] = []
    for k in range(len(results[0][:,0])):
        avg = 0
        for j in range(len(results)):
            result = results[j]
            avg = avg + result[k,1]
        avg = avg/len(results)
        avg_dirs[(0,i)].append(avg)

    color = 'tab:red'
    #axs.plot(avg_dirs, color=color)
    
for i in range(50):
    axs.plot(avg_dirs[(0,i)], color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_0mV.pdf')

In [ ]:
models = []
for i in range(50):
    models.append(keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i)))
    print('Model {} loaded'.format(i))

final_dirs[0] = []
ef_series = []
for i in range(17):
    ef_series.append(0)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0
    
fig, axs = plt.subplots()
ax2 = axs.twinx()

for i in range(50):
    model = models[i]
    print('Model {}'.format(i))

    final_dirs[0].append([])

    results = []

    for track in range(1,51):
        init = df.loc[(df['track']==track)&(df['ef']==0)&(df['set']==2)]

        result = run_sim(model, init, ef_series)
        results.append(result)
        final_dirs[0][i].append(result[-1,1])

        color = 'tab:blue'
        axs.set_ylabel('Directedness', color=color)
        #axs.plot(result[:,1], color=color)
        axs.tick_params(axis='y', labelcolor=color)

    all_res[0] = results
    
    axs.set_xlabel('Timesteps (5min/step)')
    color = 'tab:orange'
    ax2.set_title('Offline Simulation 0mV/mm')
    ax2.set_ylabel('EF (mV/mm)', color=color)
    ax2.plot(all_res[0][0][:,0]*1000, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,200])

    avg_dirs = []
    for i in range(len(results[0][:,0])):
        avg = 0
        for j in range(len(results)):
            result = results[j]
            avg = avg + result[i,1]
        avg = avg/len(results)
        avg_dirs.append(avg)

    color = 'tab:red'
    axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_0mV.pdf')

In [ ]:
final_dirs[15] = []
ef_series = []
for i in range(17):
    ef_series.append(0.015)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0
    
fig, axs = plt.subplots()
ax2 = axs.twinx()

for i in range(50):
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
    print('Model {} loaded'.format(i))

    final_dirs[15].append([])

    results = []

    for track in range(1,51):
        init = df.loc[(df['track']==track)&(df['ef']==15)&(df['set']==2)]

        result = run_sim(model, init, ef_series)
        results.append(result)
        final_dirs[15][i].append(result[-1,1])

        color = 'tab:blue'
        axs.set_ylabel('Directedness', color=color)
        #axs.plot(result[:,1], color=color)
        axs.tick_params(axis='y', labelcolor=color)
    
    all_res[15] = results
    
    axs.set_xlabel('Timesteps (5min/step)')
    color = 'tab:orange'
    ax2.set_title('Offline Simulation 15mV/mm')
    ax2.set_ylabel('EF (mV/mm)', color=color)
    ax2.plot(all_res[15][0][:,0]*1000, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,200])

    avg_dirs = []
    for i in range(len(results[0][:,0])):
        avg = 0
        for j in range(len(results)):
            result = results[j]
            avg = avg + result[i,1]
        avg = avg/len(results)
        avg_dirs.append(avg)

    color = 'tab:red'
    axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_15mV.pdf')

In [ ]:
final_dirs[30] = []
ef_series = []
for i in range(17):
    ef_series.append(0.03)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0
    
fig, axs = plt.subplots()
ax2 = axs.twinx()

for i in range(50):
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
    print('Model {} loaded'.format(i))

    final_dirs[30].append([])

    results = []

    for track in range(1,51):
        init = df.loc[(df['track']==track)&(df['ef']==30)&(df['set']==2)]

        result = run_sim(model, init, ef_series)
        results.append(result)
        final_dirs[30][i].append(result[-1,1])

        color = 'tab:blue'
        axs.set_ylabel('Directedness', color=color)
        #axs.plot(result[:,1], color=color)
        axs.tick_params(axis='y', labelcolor=color)
    all_res[30] = results
    
    axs.set_xlabel('Timesteps (5min/step)')
    color = 'tab:orange'
    ax2.set_title('Offline Simulation 30mV/mm')
    ax2.set_ylabel('EF (mV/mm)', color=color)
    ax2.plot(results[0][:,0]*1000, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,200])

    avg_dirs = []
    for i in range(len(results[0][:,0])):
        avg = 0
        for j in range(len(results)):
            result = results[j]
            avg = avg + result[i,1]
        avg = avg/len(results)
        avg_dirs.append(avg)

    color = 'tab:red'
    axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_30mV.pdf')

In [ ]:
final_dirs[50] = []
ef_series = []
for i in range(17):
    ef_series.append(0.05)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0
    
fig, axs = plt.subplots()
ax2 = axs.twinx()

for i in range(50):
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
    print('Model {} loaded'.format(i))

    final_dirs[50].append([])

    results = []

    for track in range(1,51):
        init = df.loc[(df['track']==track)&(df['ef']==50)&(df['set']==2)]

        result = run_sim(model, init, ef_series)
        results.append(result)
        final_dirs[50][i].append(result[-1,1])

        color = 'tab:blue'
        axs.set_ylabel('Directedness', color=color)
        #axs.plot(result[:,1], color=color)
        axs.tick_params(axis='y', labelcolor=color)
    all_res[50] = results
    
    axs.set_xlabel('Timesteps (5min/step)')
    color = 'tab:orange'
    ax2.set_title('Offline Simulation 50mV/mm')
    ax2.set_ylabel('EF (mV/mm)', color=color)
    ax2.plot(results[0][:,0]*1000, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,200])

    avg_dirs = []
    for i in range(len(results[0][:,0])):
        avg = 0
        for j in range(len(results)):
            result = results[j]
            avg = avg + result[i,1]
        avg = avg/len(results)
        avg_dirs.append(avg)

    color = 'tab:red'
    axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_50mV.pdf')

In [ ]:
final_dirs[75] = []
ef_series = []
for i in range(17):
    ef_series.append(0.075)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0
    
fig, axs = plt.subplots()
ax2 = axs.twinx()

for i in range(50):
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
    print('Model {} loaded'.format(i))

    final_dirs[75].append([])

    results = []

    for track in range(1,51):
        init = df.loc[(df['track']==track)&(df['ef']==75)&(df['set']==2)]

        result = run_sim(model, init, ef_series)
        results.append(result)
        final_dirs[75][i].append(result[-1,1])

        color = 'tab:blue'
        axs.set_ylabel('Directedness', color=color)
        #axs.plot(result[:,1], color=color)
        axs.tick_params(axis='y', labelcolor=color)
    all_res[75] = results
    
    axs.set_xlabel('Timesteps (5min/step)')
    color = 'tab:orange'
    ax2.set_title('Offline Simulation 75mV/mm')
    ax2.set_ylabel('EF (mV/mm)', color=color)
    ax2.plot(results[0][:,0]*1000, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,200])

    avg_dirs = []
    for i in range(len(results[0][:,0])):
        avg = 0
        for j in range(len(results)):
            result = results[j]
            avg = avg + result[i,1]
        avg = avg/len(results)
        avg_dirs.append(avg)

    color = 'tab:red'
    axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_75mV.pdf')

In [ ]:
final_dirs[100] = []
ef_series = []
for i in range(17):
    ef_series.append(0.1)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0
    
fig, axs = plt.subplots()
ax2 = axs.twinx()

for i in range(50):
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
    print('Model {} loaded'.format(i))

    final_dirs[100].append([])

    results = []

    for track in range(1,51):
        init = df.loc[(df['track']==track)&(df['ef']==100)&(df['set']==2)]

        result = run_sim(model, init, ef_series)
        results.append(result)
        final_dirs[100][i].append(result[-1,1])

        color = 'tab:blue'
        axs.set_ylabel('Directedness', color=color)
        #axs.plot(result[:,1], color=color)
        axs.tick_params(axis='y', labelcolor=color)
    all_res[100] = results
    
    axs.set_xlabel('Timesteps (5min/step)')
    color = 'tab:orange'
    ax2.set_title('Offline Simulation 100mV/mm')
    ax2.set_ylabel('EF (mV/mm)', color=color)
    ax2.plot(results[0][:,0]*1000, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,200])

    avg_dirs = []
    for i in range(len(results[0][:,0])):
        avg = 0
        for j in range(len(results)):
            result = results[j]
            avg = avg + result[i,1]
        avg = avg/len(results)
        avg_dirs.append(avg)

    color = 'tab:red'
    axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_100mV.pdf')

In [ ]:
final_dirs[200] = []
ef_series = []
for i in range(17):
    ef_series.append(0.2)

ef_series=np.array(ef_series)
ef_series = np.stack([ef_series, ef_series],1)

print('shape of series: {}'.format(ef_series.shape))

for i in range(len(ef_series)):
    ef_series[i][1] = 0
    
fig, axs = plt.subplots()
ax2 = axs.twinx()

for i in range(50):
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
    print('Model {} loaded'.format(i))

    final_dirs[200].append([])

    results = []

    for track in range(1,51):
        init = df.loc[(df['track']==track)&(df['ef']==200)&(df['set']==2)]

        result = run_sim(model, init, ef_series)
        results.append(result)
        final_dirs[200][i].append(result[-1,1])

        color = 'tab:blue'
        axs.set_ylabel('Directedness', color=color)
        #axs.plot(result[:,1], color=color)
        axs.tick_params(axis='y', labelcolor=color)
    all_res[200] = results
    
    axs.set_xlabel('Timesteps (5min/step)')
    color = 'tab:orange'
    ax2.set_title('Offline Simulation 200mV/mm')
    ax2.set_ylabel('EF (mV/mm)', color=color)
    ax2.plot(results[0][:,0]*1000, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,200])

    avg_dirs = []
    for i in range(len(results[0][:,0])):
        avg = 0
        for j in range(len(results)):
            result = results[j]
            avg = avg + result[i,1]
        avg = avg/len(results)
        avg_dirs.append(avg)

    color = 'tab:red'
    axs.plot(avg_dirs, color=color)

axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

fig.set_size_inches(14.5, 8)

plt.savefig('figures/offline_200mV.pdf')

In [ ]:
df = pd.read_csv('data/cncc_predictions.csv')
test = df[(df['set']==2)]

efs = [0,15,30,50,75,100,200]
#efs = [50]
#plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis(np.linspace(0,1,len(efs))))

fig, ax = plt.subplots()
fig.set_size_inches(14, 8) # set figure size
#for i in range(len(efs)):
    #ax.plot([0,1], [i, 2*i])

max_avs = {}
mean_avs = {}
min_avs = {}

act_means = {}
act_stds = {}
for ef in efs:
    results = all_res[ef]
    
    this_ef = test[test['ef']==ef]
    max_av = [] #max model-average dir prediction for each slice of this EF
    mean_av = [] #mean model-average dir prediction for each slice of this EF
    min_av = [] #min model-average dir prediction for each slice of this EF
    
    act_mean = [] #mean true dir value for each slice of this EF
    act_std = [] #std dev of mean true dir value for each slice of this EF
    
    for sl in range(21,38): #loop through slices where predictions are made
        this_slice = this_ef[this_ef['slice']==sl]
        
        act_mean.append(statistics.mean(this_slice['cum_dir']))
        act_std.append(statistics.stdev(this_slice['cum_dir']))
        
        all_avs = [] #list of the average predicted directedness for this ef+slice for each model
        for i in range(50): #loop through the 50 models
            this_model = this_slice['pred_dir{}'.format(i)].dropna()
            all_avs.append(statistics.mean(this_model))
        
        max_av.append(max(all_avs))
        mean_av.append(statistics.mean(all_avs))
        min_av.append(min(all_avs))
        
    max_avs[ef] = max_av
    mean_avs[ef] = mean_av
    min_avs[ef] = min_av
    act_means[ef] = act_mean
    act_stds[ef] = act_std
    
colors = ['#1e14de','#1873db','#2be3e0','#25c460','#ede72b','#ebba28','#db3627']

lines = []
for i in range(len(efs)):
    ef = efs[i]
    #max_plot = plt.plot(range(21,38),max_avs[ef], color='k')
    line, = plt.plot(range(21,38),mean_avs[ef], color=colors[i])
    lines.append(line)
    #min_plot = plt.plot(range(21,38),min_avs[ef], color='k')
    plt.fill_between(range(21,38),min_avs[ef],max_avs[ef], color=colors[i], alpha=0.3)
    
    plt.plot(range(21,38),act_means[ef], color=colors[i], linestyle='--') #actual mean
ax.legend(lines, map('{}mV/mm'.format,efs))
ax.set_xlabel('Timestep')
ax.set_ylabel('Directedness')
ax.set_title('Average Ground Truth vs Predicted Directedness By Timestep')
        
plt.show()

In [ ]:
print(final_dirs[100][48])

In [ ]:
dirs_list = []
for ef in [0,15,30,50,75,100,200]:
    thisbox = []
    for i in range(50):
        #thisbox.append(sum(final_dirs[ef][i])/len(final_dirs[ef][i]))
        for direction in final_dirs[ef][i]:
            thisbox.append(direction)
    dirs_list.append(thisbox)
plt.boxplot(dirs_list)

In [ ]:
real_finals = []
for ef in [0,15,30,50,75,100,200]:
    real_finals.append(df[(df['set']==1) & (df['slice']==37) & (df['ef']==ef)]['cum_dir'])

plt.boxplot(real_finals)

In [ ]:
sim_means = []
sim_devs = []
real_means = []
real_devs = []
for i in range(7):
    sim_means.append(statistics.mean(dirs_list[i]))
    sim_devs.append(stats.sem(dirs_list[i]))
    real_means.append(statistics.mean(real_finals[i]))
    real_devs.append(stats.sem(real_finals[i]))

sim_pos = []
real_pos = []
for i in range(7):
    real_pos.append(2*i-0.4)
    sim_pos.append(2*i+0.4)
    
fig, ax = plt.subplots()
fig.set_size_inches(14.5, 10)
ax.bar(real_pos, real_means, yerr=real_devs)
ax.bar(sim_pos, sim_means, yerr=sim_devs)
ax.legend(['Ground Truth','Simulation'])

In [ ]:
models = []
for i in range(50):
    models.append(keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i)))
    print('Model {} loaded'.format(i))

In [ ]:
efs = [0,15,30,50,75,100,200]
for ef in efs:
    print('Simulating with constant EF at {}mV/mm'.format(ef))
    final_dirs[ef] = []
    
    ef_series = []
    for i in range(17):
        ef_series.append(ef/1000)

    ef_series=np.array(ef_series)
    ef_series = np.stack([ef_series, np.zeros(ef_series.shape)],1)

    print('shape of series: {}'.format(ef_series.shape))

    i=0
    for model in models:
        print('Model {}'.format(i))
        final_dirs[ef].append([])

        results = []

        for track in range(1,51):
            init = df.loc[(df['track']==track)&(df['ef']==ef)&(df['set']==2)]

            result = run_sim(model, init, ef_series)
            results.append(result)
            final_dirs[0][i].append(result[-1,1])
            
            for j in range(len(result)):
                df.loc[(df['track']==track)&(df['ef']==ef)&(df['set']==2)&(df['slice']==21+j),'sim{}'.format(i)] = result[j,1]
        
        i+=1
    
    print(df.head(40))

In [ ]:
efs = [0,15,30,50,75,100,200]
for i in range(50):
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
    print('Model {}'.format(i))
    for ef in efs:
        print('Simulating with constant EF at {}mV/mm'.format(ef))

        ef_series = []
        for _ in range(17):
            ef_series.append(ef/1000)

        ef_series=np.array(ef_series)
        ef_series = np.stack([ef_series, np.zeros(ef_series.shape)],1)

        print('shape of series: {}'.format(ef_series.shape))

        results = []

        for track in range(1,51):
            init = df.loc[(df['track']==track)&(df['ef']==ef)&(df['set']==2)]

            result = run_sim(model, init, ef_series)
            results.append(result)

            for j in range(len(result)-20):
                df.loc[(df['track']==track)&(df['ef']==ef)&(df['set']==2)&(df['slice']==21+j),'sim{}'.format(i)] = result[j+20,1]

    print('tail of sim{}'.format(i))
    print(df['sim{}'.format(i)].tail(40))

In [ ]:
df.head()

In [ ]:
print('Writing to CSV')
df.to_csv('data/sims.csv'.format(ef), index=False)

In [ ]:
efs = [0,15,30,50,75,100,200]
for ef in efs:
    print('EF: {}'.format(ef))
    final_dirs[ef] = []
    ef_series = []
    for i in range(17):
        ef_series.append(ef/1000)

    ef_series=np.array(ef_series)
    ef_series = np.stack([ef_series, ef_series],1)

    print('shape of series: {}'.format(ef_series.shape))

    for i in range(len(ef_series)):
        ef_series[i][1] = 0

    fig, axs = plt.subplots()
    ax2 = axs.twinx()

    for i in range(50):
        model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
        print('Model {} loaded'.format(i))

        final_dirs[ef].append([])

        results = []

        for track in range(1,51):
            init = df.loc[(df['track']==track)&(df['ef']==ef)&(df['set']==2)]

            result = run_sim(model, init, ef_series)
            results.append(result)
            final_dirs[ef][i].append(result[-1,1])

            color = 'tab:blue'
            axs.set_ylabel('Directedness', color=color)
            #axs.plot(result[:,1], color=color)
            axs.tick_params(axis='y', labelcolor=color)

        all_res[ef] = results

        axs.set_xlabel('Timesteps (5min/step)')
        color = 'tab:orange'
        ax2.set_title('Offline Simulation {}mV/mm'.format(ef))
        ax2.set_ylabel('EF (mV/mm)', color=color)
        ax2.plot(all_res[ef][0][:,0]*1000, color=color)
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.set_ylim([0,200])

        avg_dirs[(ef,i)] = []
        for k in range(len(results[0][:,0])):
            avg = 0
            for j in range(len(results)):
                result = results[j]
                avg = avg + result[k,1]
            avg = avg/len(results)
            avg_dirs[(ef,i)].append(avg)

        color = 'tab:red'
        #axs.plot(avg_dirs, color=color)

    for i in range(50):
        axs.plot(avg_dirs[(ef,i)], color=color)

    axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

    fig.set_size_inches(14.5, 8)

    plt.savefig('figures/offline_{}mV.pdf'.format(ef))

In [ ]:
print(avg_dirs[(100,1)])

In [ ]:
fig, ax = plt.subplots()

dirs_list = []
for ef in [0,15,30,50,75,100,200]:
    thisbox = []
    for i in range(50):
        for direction in final_dirs[ef][i]:
            thisbox.append(direction)
    dirs_list.append(thisbox)
sim_boxes = ax.boxplot(dirs_list,positions=np.array([1,2,3,4,5,6,7])*2+0.25,showfliers=False,showmeans=True,patch_artist=True)

real_finals = []
for ef in [0,15,30,50,75,100,200]:
    real_finals.append(df[(df['set']==2) & (df['slice']==37) & (df['ef']==ef)]['cum_dir'])

real_boxes = ax.boxplot(real_finals,positions=np.array([1,2,3,4,5,6,7])*2-0.25,showfliers=False,showmeans=True,patch_artist=True)

#styling
fig.set_size_inches(9, 5)
ax.set_title('Final Directedness Distributions for Test Set Cells')
ax.set_xlabel('EF Strength')
ax.set_ylabel(r'Final Directedness ($\cos(\theta)$)')
plt.xticks(range(2,16,2), labels=['{}mV/mm'.format(ef) for ef in efs])

#colors
sim_color = 'lightblue'
real_color = 'firebrick'
for patch in sim_boxes['boxes']:
    patch.set_facecolor(sim_color)
for patch in real_boxes['boxes']:
    patch.set_facecolor(real_color)

#legend information
base_label, = ax.plot([],'s',color=real_color)
inter_label, = ax.plot([],'s',color=sim_color)
median_label, = ax.plot([],color='orange')
mean_label, = ax.plot([],'^',color='green')
ax.legend(
    [base_label,inter_label,median_label,mean_label],
    ['Ground Truth','Base Model Simulations','Median','Mean'],
    fancybox=True, shadow=True)

plt.savefig('figures/sim_finaldirs.pdf') #save the figure

real_means = []
sim_means = []
real_meds = []
sim_meds = []
for r in real_finals:
    real_means.append(np.mean(r))
    real_meds.append(np.median(r))
for d in dirs_list:
    sim_means.append(np.mean(d))
    sim_meds.append(np.median(d))

print('Real means:',real_means)
print('Sim means:',sim_means)
print('Real medians:',real_meds)
print('Sim medians:',sim_meds)

print('Correlation between means:',np.corrcoef(real_means,sim_means))
print('Correlation between medians:',np.corrcoef(real_meds,sim_meds))

In [ ]:
real_finals = []
for ef in [0,15,30,50,75,100,200]:
    real_finals.append(df[(df['set']==1) & (df['slice']==37) & (df['ef']==ef)]['cum_dir'])

plt.boxplot(real_finals)

In [ ]:
df = pd.read_csv('data/sims.csv')
test = df[(df['set']==2)]

efs = [0,15,30,50,75,100,200]
#efs = [50]
#plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis(np.linspace(0,1,len(efs))))

fig, ax = plt.subplots()
fig.set_size_inches(14, 8) # set figure size
#for i in range(len(efs)):
    #ax.plot([0,1], [i, 2*i])

max_avs = {}
mean_avs = {}
min_avs = {}

act_means = {}
act_stds = {}
for ef in efs:
    this_ef = test[test['ef']==ef]
    max_av = [] #max model-average dir prediction for each slice of this EF
    mean_av = [] #mean model-average dir prediction for each slice of this EF
    min_av = [] #min model-average dir prediction for each slice of this EF
    
    act_mean = [] #mean true dir value for each slice of this EF
    act_std = [] #std dev of mean true dir value for each slice of this EF
    
    for sl in range(21,38): #loop through slices where predictions are made
        this_slice = this_ef[this_ef['slice']==sl]
        
        act_mean.append(statistics.mean(this_slice['cum_dir']))
        act_std.append(statistics.stdev(this_slice['cum_dir']))
        
        all_avs = [] #list of the average predicted directedness for this ef+slice for each model
        for i in range(50): #loop through the 50 models
            this_model = this_slice['sim{}'.format(i)].dropna()
            all_avs.append(statistics.mean(this_model))
        
        max_av.append(max(all_avs))
        mean_av.append(statistics.mean(all_avs))
        min_av.append(min(all_avs))
        
    max_avs[ef] = max_av
    mean_avs[ef] = mean_av
    min_avs[ef] = min_av
    act_means[ef] = act_mean
    act_stds[ef] = act_std
    
colors = ['#1e14de','#1873db','#2be3e0','#25c460','#ede72b','#ebba28','#db3627']

lines = []
for i in range(len(efs)):
    ef = efs[i]
    #max_plot = plt.plot(range(21,38),max_avs[ef], color=colors[i])
    line, = plt.plot(range(21,38),mean_avs[ef], color=colors[i])
    lines.append(line)
    #min_plot = plt.plot(range(21,38),min_avs[ef], color=colors[i])
    plt.fill_between(range(21,38),min_avs[ef],max_avs[ef], color=colors[i], alpha=0.3)
    
    plt.plot(range(21,38),act_means[ef], color=colors[i], linestyle='--') #actual mean
ax.legend(lines, map('{}mV/mm'.format,efs))
ax.set_xlabel('Timestep')
ax.set_ylabel('Directedness')
ax.set_title('Average Ground Truth vs Predicted Directedness By Timestep')
        
plt.show()

### Simulation for more timesteps

In [ ]:
all_res = {}
avg_dirs = {}
for ef in [0,15,30,50,75,100,200]:
    ef_series = []
    for i in range(250):
        ef_series.append(ef/1000)

    ef_series=np.array(ef_series)
    ef_series = np.stack([ef_series, ef_series],1)

    print('shape of series: {}'.format(ef_series.shape))

    for i in range(len(ef_series)):
        ef_series[i][1] = 0

    fig, axs = plt.subplots()
    ax2 = axs.twinx()

    for i in range(50):
        model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(i))
        print('Model {} loaded'.format(i))

        results = []

        for track in range(1,51):
            init = df.loc[(df['track']==track)&(df['ef']==ef)&(df['set']==2)]

            result = run_sim(model, init, ef_series)
            results.append(result)

        all_res[ef] = results

        axs.set_xlabel('Timesteps (5min/step)')
        color = 'tab:orange'
        ax2.set_title('Offline Simulation {}mV/mm'.format(ef))
        ax2.set_ylabel('EF (mV/mm)', color=color)
        ax2.plot(results[0][:,0]*1000, color=color)
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.set_ylim([0,200])

        avg_dirs = []
        for i in range(len(results[0][:,0])):
            avg = 0
            for j in range(len(results)):
                result = results[j]
                avg = avg + result[i,1]
            avg = avg/len(results)
            avg_dirs.append(avg)

        color = 'tab:red'
        axs.plot(avg_dirs, color=color)
    axs.set_ylabel('Directedness', color=color)
    axs.tick_params(axis='y', labelcolor=color)
        
    axs.plot((20,20),(-1,1),color='#4b0082',linewidth=2)

    fig.set_size_inches(14.5, 8)

    plt.savefig('figures/offline_{}mV_250steps.pdf'.format(ef))